In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from collections import Counter
from sklearn.utils import resample
from sklearn import preprocessing
from scipy.sparse import csr_matrix

In [ ]:
# Read the file and store the data
def readTrainfile(filepath, sep):
    read_data = pd.read_csv(filepath, sep=sep, encoding = 'latin-1')
    return read_data

# Save the file the output file
def saveOutput(filePath, data):
    # writing to .txt
    np.savetxt(filePath, data, fmt='%s')

# Save the data to csv
def saveCSV(data,namefile):
    data.to_csv(namefile, encoding='utf-8', index=False)

In [ ]:
FinalTest =readTrainfile(filepath="/test.dat", sep= ' ')
train= readTrainfile(filepath="/train.dat", sep= ' ')

In [ ]:
train

,userID,movieID,rating
0,75,3,1.0
1,75,32,4.5
2,75,110,4.0
3,75,163,4.0
4,75,165,4.5
...,...,...,...
641694,71534,42900,4.0
641695,71534,44555,4.0
641696,71534,46578,4.0
641697,71534,61075,5.0


In [ ]:
FinalTest

,userID,movieID
0,75,653
1,75,1527
2,75,3258
3,75,7007
4,78,82
...,...,...
71294,71534,2208
71295,71534,2997
71296,71534,4306
71297,71534,7132


In [ ]:
from sklearn.model_selection import KFold

# Creating folds, with N as 3. 
kf = KFold(n_splits=3, shuffle=True)
kf.get_n_splits(train)

train_X = train.drop(columns=['rating'])
train_Y = train['rating']

for train_index, test_index in kf.split(train_X, train_Y):
  # print(train_index, test_index)
    X_train, X_test = train_X.iloc[train_index], train_X.iloc[test_index]
    y_train, y_test = train_Y[train_index], train_Y[test_index]

    break

In [ ]:
# preprocessTrainData=train.dropna()
# training_set_X, training_set_Y= train.drop(columns=['rating']), train['rating']
# training_set_Y

In [ ]:
from xgboost import XGBRegressor

preprocessTrainData=train.dropna()
training_set_X, training_set_Y= preprocessTrainData.drop(columns=['rating']), preprocessTrainData['rating']

params = {'subsample': 0.5, 'n_estimators': 5000, 'max_depth': 20, 'learning_rate': 0.2, 'colsample_bytree': 0.8, 'colsample_bylevel': 0.9}

xgb_model = XGBRegressor(**params)
xgb_model.fit(training_set_X, training_set_Y)

y_pred = xgb_model.predict(FinalTest)
y_pred

[20:43:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


array([2.984791 , 3.6664104, 2.8757107, ..., 4.0088367, 4.3278427,
       3.8136973], dtype=float32)

In [ ]:
saveOutput("/xgb_model_basic+params.txt",y_pred)

In [ ]:
from xgboost import XGBRegressor

params = {'subsample': 0.5, 'n_estimators': 5000, 'max_depth': 20, 'learning_rate': 0.2, 'colsample_bytree': 0.8, 'colsample_bylevel': 0.9}

xgb_model = XGBRegressor(**params)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)
y_pred

from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred, squared=False)

[21:04:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.8132638903399342

In [ ]:
from xgboost import XGBRegressor

params = {'subsample': 0.5, 'n_estimators': 1000, 'max_depth': 20, 'learning_rate': 1.0, 'colsample_bytree': 0.8, 'colsample_bylevel': 0.9}

xgb_model = XGBRegressor(**params)
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)
y_pred

from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred, squared=False)

[20:24:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.8220505765420413

In [ ]:
from xgboost import XGBClassifier

params = {'subsample': 0.5, 'n_estimators': 50, 'max_depth': 5, 'learning_rate': 1.0, 'colsample_bytree': 0.8, 'colsample_bylevel': 0.9}

xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)
y_pred

from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred, squared=False)

1.1314235345732628

In [ ]:
# saveOutput("/xgb_model_basic+1000.txt",y_pred)

In [ ]:
rounder = lambda x: round(x/0.5) * 0.5
vfunc = np.vectorize(rounder)
output5round = vfunc(y_pred)

In [ ]:
saveOutput("/xgb_model_basic+1000+rounded.txt",output5round)